**1. Install and Import Dependencies**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


**2. Instantiate Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

**3. Encode and Calculate Sentiment**

In [ ]:
tokens = tokenizer.encode('It was good, but could have been better, Great', return_tensors='pt')

In [ ]:
tokens

tensor([[  101, 10197, 10140, 12050,   117, 10502, 12296, 10574, 10662, 16197,
           117, 11838,   102]])

In [ ]:
result = model(tokens)

In [ ]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.4087, -0.3734,  2.0270,  1.6085, -0.7734]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
int(torch.argmax(result.logits)) + 1

3

**4. Collect Reviews**

In [ ]:
r = requests.get('https://www.yelp.com/biz/sushi-you-oakland-gardens-3?osq=sushi+you')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [results.text for results in results]

In [ ]:
reviews[0]

"what a hidden gem in Bayside - if you want fresh sushi and amazing lunch special deals this is the spot for you! anytime I am craving a sushi roll this is my go to spot as they have a ton of different rolls to choose from and the lunch special where you can pick two or three rolls is an amazing deal. They have limited seating inside in a while. I've never actually ate inside myself (i usually order pick up to go) there's always people inside eating and would love to actually try it one day. The staff is super friendly and you can see the chef preparing the fish in the counter right in front of you. Typically I pick up the food is ready fairly quickly and I've never had to wait a really long time to get my food. This spot is absolutely worth it and will always come back here!"

**5. Load Reviews into DataFrame and Score**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df['review'].iloc[0]

"what a hidden gem in Bayside - if you want fresh sushi and amazing lunch special deals this is the spot for you! anytime I am craving a sushi roll this is my go to spot as they have a ton of different rolls to choose from and the lunch special where you can pick two or three rolls is an amazing deal. They have limited seating inside in a while. I've never actually ate inside myself (i usually order pick up to go) there's always people inside eating and would love to actually try it one day. The staff is super friendly and you can see the chef preparing the fish in the counter right in front of you. Typically I pick up the food is ready fairly quickly and I've never had to wait a really long time to get my food. This spot is absolutely worth it and will always come back here!"

In [ ]:
df['review'].iloc[1]

'Good deal for lunch special.  3 rolls for 15$ and Congress with soup.  My favorites are the eel avocado, salmon and spicy tuna rolls.  Special rolls like the shrimp tempura is a good choice if you want some crunch.Best to order online for pickup if ordering to go.  Metered parking on main Street or free parking around the corner where the residential apartments are.'

In [ ]:
df.head()

,review
0,what a hidden gem in Bayside - if you want fre...
1,Good deal for lunch special. 3 rolls for 15$ ...
2,"Good spot for Sushi rolls in oakland, bayside ..."
3,"Good deal for 3 rolls for $15. Got the EEl, ..."
4,Warm welcoming staff & owner. Fish selection ...


In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [ ]:
sentiment_score(df['review'].iloc[0])

5

In [ ]:
sentiment_score(df['review'].iloc[1])

4

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,what a hidden gem in Bayside - if you want fre...,5
1,Good deal for lunch special. 3 rolls for 15$ ...,4
2,"Good spot for Sushi rolls in oakland, bayside ...",4
3,"Good deal for 3 rolls for $15. Got the EEl, ...",4
4,Warm welcoming staff & owner. Fish selection ...,3
5,Ordered a party tray for Mother's Day dinner. ...,3
6,Always good price! We order once a week. Their...,5
7,What a gem! Service was very friendly and acco...,5
8,My go to sushi spot when I'm passing bayside. ...,5
9,My go to Sushi spot in the neighborhood becaus...,5
